In [177]:
#importing libraries
import numpy as np
import pandas as pd
import functools as ft

In [178]:
#reading the csv file
df= pd.read_csv("C:/Users/rajaa/OneDrive/Desktop/NYPD_Complaint_Data_Historic.csv")

C:\Users\rajaa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [179]:
#dropping the irrelevant columns
toDrop= ['CMPLNT_NUM','CMPLNT_FR_DT','CMPLNT_FR_TM','CMPLNT_TO_DT','CMPLNT_TO_TM','KY_CD','PD_CD','PD_DESC','CRM_ATPT_CPTD_CD'
         ,'LAW_CAT_CD','ADDR_PCT_CD','LOC_OF_OCCUR_DESC','PREM_TYP_DESC','PARKS_NM','HADEVELOPT','X_COORD_CD','Y_COORD_CD','Latitude','Longitude','Lat_Lon']

df.drop(toDrop, inplace=True, axis= 1)

In [180]:
#creating a column named year reffering to year of occurence
df['year']=df.RPT_DT.str.slice(-4)    #will consider the last 4 characters of the RPT_DT

In [181]:
#updating the values of the cells of RPT_DT referring to Month only
df['RPT_DT']= df.RPT_DT.str.slice(0,2)

#if there is month sliced as '5/' then / will be replaced with empty string
df['RPT_DT']=df.RPT_DT.str.replace("/","")

In [182]:
#changing the data typy of RPT_DT and year from object to Year
newType= {'RPT_DT': int, 'year': int}
df=df.astype(newType)

In [183]:
#creating a list containg the unique numbers of the year and month mentioned
years= pd.unique(df['year'])
months= pd.unique(df['RPT_DT'])

In [184]:
#renaming the column RPT_DT to Month and year to Year respectively
df= df.rename(columns= {'RPT_DT': 'Month', 'year': 'Year', 'OFNS_DESC': 'Category'})

In [185]:

#to retrieve data frame for all years on monthly basis crime frequency
def getYearDataFrame(NoOfYears):
    yearList= list(np.arange(NoOfYears))
    for i in np.arange(NoOfYears):
        yearList[i]= yearDataFrame(years[i])
    return yearList

#to retrieve data frame for monthly basis crime freqeuncy for a single year on the basis of the given year
def yearDataFrame(yearNumber):
    yearTemp= df.loc[df['Year']==yearNumber]
    monthFrame= getMonthDataFrame(yearTemp,len(months),yearNumber)
    yearDf= ft.reduce(lambda left, right: pd.merge(left,right,on='Category',how='outer'),monthFrame)
    yearDf=yearDf.fillna(0)
    return yearDf

#to retrive the monthly basis crime frequency for all months of a year
def getMonthDataFrame(yearTem,noOfMonths,yearNum):
    monthList= list(np.arange(noOfMonths))
    for i in np.arange(noOfMonths):
        monthList[i]= monthDataFrame(yearTem,months[i],yearNum)
    return monthList

#to retrieve the dataframe for crime frequency of a single month
def monthDataFrame(yearTempo,monthNumber,yearNo):
    monthTemp= yearTempo.loc[df['Month']==monthNumber]
    monthlyTempDf= monthTemp['Category'].value_counts().rename_axis('Category').reset_index(name="Month-"+str(monthNumber)+"("+str(yearNo)+")")
    return monthlyTempDf    

final_list= getYearDataFrame(len(years))
    

In [186]:
#merging all the year based dataframe into a single dataframe taking the Category as the common field
df_final= ft.reduce(lambda left, right: pd.merge(left,right,on='Category',how='outer'),final_list)

#replace all na values with 0
df_final= df_final.fillna(0)

In [187]:
#adding row showing total crime in each month
crimeInMonth= np.arange((len(months)*len(years))+1)
count=0
for i in np.arange(len(years)):
    for j in np.arange(len(months)):
               #crimeInMonth[count+1]= np.sum(df_final['Month-12(2015)'])
                crimeInMonth[count+1]= np.sum(df_final['Month-'+str(months[j])+"("+str(years[i])+')'])
                count+=1
               # crimeInMonth[count+1]= np.sum(df_final['Month-'+str(months[j])+"("+str(years[i])+")"])
    

crimeMonth= pd.DataFrame([crimeInMonth],columns=df_final.columns)
crimeMonth['Category']='TOTAL'

df_final= pd.concat([df_final,crimeMonth],ignore_index=True)

In [175]:
#converting the final dataframe into csv file
df_final.to_csv("check1.csv")